## Data Handling

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [13]:
df = pd.read_csv('../../../../../../jaeyeun/01_nh_poc/16_re_label/split_70_15_15_re_3_prep_v10.csv')

df.head()

,file_name,raw_text,dataset_n,label_1,label_2,label_3,label_4,train_val_test,prep_v10_text
0,D1512065.txt,보 도 자 료\nhttp://www.motie.go.kr 2015년 12월28일(월...,0,0,0,0,0,0,가능N 국가N 기술N 표준원N 기술N 규제N 조정N 오광N 과장N 연구사N 기업N ...
1,R2002420.txt,보 도 자 료 작성과 공공데이터정책과\n2020년 2월 13일(목) 조간 (2. 1...,0,0,0,0,0,0,공공N 데이터N 정책N 이후N 남호N 공공N 데이터N 청년N 인턴N 참가N 청년N ...
2,R2002370.txt,2020년 2월 11일(화) 석간부터 보도하여 주시기 바랍니다. * 통신․방송․인터...,0,0,0,0,0,0,가능N 벤처N 투자N 과장N 벤처N 혁신N 정책N 과장N 벤처N 생태N 민간N 중심...
3,R2004373.txt,보도참고자료\nhttp://www.motie.go.kr 배포 즉시 보도하여 주시기 ...,0,0,0,0,0,0,즉시N 기정N 통부N 연구N 개발N 정책N 판식N 정보N 기술N 정책N 연구N 제도...
4,D1506066.txt,보 도 자 료\n보도일시 2015. 6. 12.(금) 조간(온라인 6. 11. 12...,0,0,0,0,0,0,온라인N 이후N 디지털N 정책N 과장N 지운N 스마트미디어N 엑스N 캠프N 신규N ...


* only for kdi dataset(exp1) & label_4

In [16]:
df = df[df['dataset_n'] == 0]

In [17]:
X_train_df = df[df['train_val_test'] == 0]
X_val_df = df[df['train_val_test'] == 1]
X_test_df = df[df['train_val_test'] == 2]
y_train = df[df['train_val_test'] == 0]['label_4']
y_val = df[df['train_val_test'] == 1]['label_4']
y_test = df[df['train_val_test'] == 2]['label_4']

In [18]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(7555, 9)
(1621, 9)
(1624, 9)
(7555,)
(1621,)
(1624,)


In [19]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

* data is 'prep_v10_text'

In [20]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v10_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [21]:
X_train = X_train_tfidf.transform(X_train_df['prep_v10_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v10_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v10_text'])

In [22]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7555, 100000)
(1621, 100000)
(1624, 100000)


## Classify

* LightGBM RandomSearch

In [23]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [25]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                   'device': ['gpu'],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "multi_error",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed: 24.2min remaining: 26.2min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed: 29.7min remaining: 20.6min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed: 32.4min remaining: 13.9min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed: 36.8min remaining:  8.6min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 39.4min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 43.7min finished


[1]	training's multi_error: 0.345731	training's multi_logloss: 1.94319	valid_1's multi_error: 0.383714	valid_1's multi_logloss: 1.96544
Training until validation scores don't improve for 20 rounds
[2]	training's multi_error: 0.19775	training's multi_logloss: 1.72218	valid_1's multi_error: 0.241826	valid_1's multi_logloss: 1.76112
[3]	training's multi_error: 0.147055	training's multi_logloss: 1.54963	valid_1's multi_error: 0.202961	valid_1's multi_logloss: 1.60478
[4]	training's multi_error: 0.124686	training's multi_logloss: 1.40867	valid_1's multi_error: 0.183837	valid_1's multi_logloss: 1.47832
[5]	training's multi_error: 0.111979	training's multi_logloss: 1.29008	valid_1's multi_error: 0.172116	valid_1's multi_logloss: 1.3729
[6]	training's multi_error: 0.104699	training's multi_logloss: 1.18604	valid_1's multi_error: 0.166564	valid_1's multi_logloss: 1.28096
[7]	training's multi_error: 0.0987426	training's multi_logloss: 1.09598	valid_1's multi_error: 0.167798	valid_1's multi_loglo

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'device': ['gpu'],
                                        'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [26]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [42]:
np.argsort(lgbm_random.best_estimator_.predict_proba(X_test)[0])[-2:]

array([8, 0])

In [32]:
pred_test

array([0, 0, 0, ..., 9, 9, 9])

In [43]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.9936465916611515
0.8778531770512029
0.8854679802955665


In [45]:
import pickle
with open('./LightGBM_exp1_prep10_label4.txt', 'wb') as f:
    pickle.dump(lgbm_random, f)

In [28]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,device,mean_test_score
0,0.6,1,3,0.010,gpu,0.790470
0,0.8,5,3,0.015,gpu,0.803441
0,0.9,7,5,0.010,gpu,0.821443
0,0.8,5,9,0.010,gpu,0.835738
0,1.0,5,15,0.015,gpu,0.842356
0,0.8,3,5,0.025,gpu,0.845400
0,0.6,1,5,0.025,gpu,0.845797
0,0.6,7,25,0.025,gpu,0.847386
0,0.9,7,15,0.025,gpu,0.848048
0,0.8,7,15,0.025,gpu,0.849107
